In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/NIR/ods_broadcasts_201905301157.csv.zip" /content/ods_broadcasts_201905301157.csv.zip
!cp "/content/drive/MyDrive/NIR/ods_match_news.csv.zip" /content/ods_match_news.csv.zip
!unzip /content/ods_broadcasts_201905301157.csv.zip
!unzip /content/ods_match_news.csv.zip

!cp "/content/drive/MyDrive/NIR/predicted_data.zip" /content/predicted_data.zip
!unzip /content/predicted_data.zip

Archive:  /content/ods_broadcasts_201905301157.csv.zip
  inflating: ods_broadcasts_201905301157.csv  
Archive:  /content/ods_match_news.csv.zip
  inflating: ods_match_news.csv      
Archive:  /content/predicted_data.zip
  inflating: predicted_data.csv      


In [ ]:
!pip install -U --quiet sentence-transformers evaluate nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 17.9 MB/s eta 0:00:

In [ ]:
import pandas as pd
import re
import numpy as np
import nltk
import datetime as dt
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data_broadcasts = pd.read_csv("/content/ods_broadcasts_201905301157.csv")

def remove_tags(html):
    soup = BeautifulSoup(str(html), "html.parser")
    
    for data in soup(['style', 'script']):
        data.decompose()

    return ' '.join(soup.stripped_strings)

data_broadcasts['content'] = data_broadcasts['content'].apply(remove_tags)

data_broadcasts['content'] = data_broadcasts['minute'].apply(lambda x: str(x)) + " минута. " + data_broadcasts['content']
data_broadcasts['content'] = data_broadcasts['content'].apply(lambda x: re.sub(r'[г]+[о]+[л]+', 'гол', x))

data_broadcasts['match_time'] = data_broadcasts['match_time'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
data_broadcasts['match_time'] = pd.to_datetime(data_broadcasts['match_time'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

data_broadcasts = data_broadcasts.sort_values(['match_id', 'minute', 'message_time'], ascending = True)
data_broadcasts

<ipython-input-5-548401de5d80>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(str(html), "html.parser")
<ipython-input-5-548401de5d80>:4: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(str(html), "html.parser")


,match_id,team1,team2,name,match_time,type,minute,content,message_time
0,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,0 минута. Добрый день! Sports.ru поздравляет в...,1237034763
1,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,"0 минута. Будем надеяться, что признаков пресл...",1237034996
2,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,"0 минута. Что ж, сегодняшних соперников можно ...",1237035202
3,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,0 минута. Зимой подмосковный клуб едва не исче...,1237035415
4,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,"0 минута. ""Локомотив"" зимой старался стать луч...",1237035796
...,...,...,...,...,...,...,...,...,...
722062,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,empty,63,"63 минута. 2:1 - ""Питтсбург"" выигрывает второй...",1464837758
722063,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,empty,63,"63 минута. Очевидно, что залог успеха ""Питтсбу...",1464837879
722064,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,empty,63,"63 минута. Другими словами, ДеБур должен в пер...",1464838029
722065,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,empty,63,63 минута. Большое спасибо за чтение! До новых...,1464838220


In [ ]:
data_news = pd.read_csv('/content/ods_match_news.csv')

# Удалим новости, которые не содержат текста
data_news.dropna(axis=0, how='any', subset=['body'], inplace=True)

data_news['name'] = data_news['name'].apply(remove_tags)
data_news['body'] = data_news['body'].apply(remove_tags)

data_news['body'] = data_news['body'].apply(lambda x: str(x).replace(' \x96', ""))

size_body = 80
data_news['part_body'] =  data_news['name'] +  '. ' + data_news['body'].apply(lambda x: ' '.join(x.split()[:size_body]))

data_news['ctime'] = pd.to_datetime(data_news['ctime'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

data_news

<ipython-input-5-548401de5d80>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(str(html), "html.parser")


,id,name,ctime,body,match_id,part_body
0,7005921,Зико: «Задача может быть только одна: пройти в...,2009-02-18 01:06:26,Главный тренер ЦСКА Зико поделился мыслями о п...,18936,Зико: «Задача может быть только одна: пройти в...
1,7005943,"Текстовые онлайны матчей «Зенит» – «Штутгарт»,...",2009-02-18 01:08:20,Сегодня на Sports.ru дебютируют текстовые онла...,18935,"Текстовые онлайны матчей «Зенит» – «Штутгарт»,..."
2,7005943,"Текстовые онлайны матчей «Зенит» – «Штутгарт»,...",2009-02-18 01:08:20,Сегодня на Sports.ru дебютируют текстовые онла...,18936,"Текстовые онлайны матчей «Зенит» – «Штутгарт»,..."
3,7006022,Стилян Петров: «ЦСКА выиграл все матчи в этом ...,2009-02-18 01:21:55,Полузащитник «Астон Виллы» Стилян Петров подел...,18936,Стилян Петров: «ЦСКА выиграл все матчи в этом ...
4,7006398,Хиддинк постетит матч «Астон Виллы» и ЦСКА,2009-02-18 03:17:23,Главный тренер сборной России и «Челси» Гус Хи...,18936,Хиддинк постетит матч «Астон Виллы» и ЦСКА. Гл...
...,...,...,...,...,...,...
92992,1040645221,Петр Чех: Если бы пропустил от Денисова — это ...,2016-06-02 14:44:24,Голкипер сборной Чехии Петр Чех после победы в...,1095139,Петр Чех: Если бы пропустил от Денисова — это ...
92993,1040648050,"Конор Шери: «Кросби сказал, что выиграет вбрас...",2016-06-02 16:17:07,Форвард «Питтсбурга» Конор Шери прокомментиров...,1104866,"Конор Шери: «Кросби сказал, что выиграет вбрас..."
92994,1040648436,Сидни Кросби: «Победным голом мы больше обязан...,2016-06-02 16:29:35,Капитан «Питтсбурга» Сидни Кросби прокомментир...,1104866,Сидни Кросби: «Победным голом мы больше обязан...
92995,1040653602,Джастин Браун: «Питтсбург» все наступал и наст...,2016-06-02 19:36:29,Защитник «Сан-Хосе» Джастин Браун оценил ход с...,1104866,Джастин Браун: «Питтсбург» все наступал и наст...


In [ ]:
predicted_data = pd.read_csv("/content/predicted_data.csv")

relevant_match_ids = predicted_data['match_id'].drop_duplicates().tolist()
relevant_news_ids = predicted_data['id'].drop_duplicates().tolist()

relevant_data_broadcasts = data_broadcasts[data_broadcasts['match_id'].isin(relevant_match_ids)].reset_index().drop(['index'], axis=1)
relevant_data_news = data_news[data_news['id'].isin(relevant_news_ids)].reset_index().drop(['index'], axis=1)

In [ ]:
content_type = [
    'attention',
    'zamena',
    'yellowcard',
    'ball',
    'puck',
    'penalty',
    'redcard',
    'gk_zamena',
    'fight',
    '2points',
    '3points',
    'freethrow2',
    'freethrow1',
    'timeout',
    'delete'
]

In [ ]:
df = {}
count_df_row = 0
size_window_before = 1
size_window_after = 3

for index in range(len(relevant_data_broadcasts)):
  current_row = relevant_data_broadcasts.loc[index]
  # все события кроме attention
  if current_row['type'] in content_type[1:]:
    current_minute = current_row['minute']
    
    window_data = relevant_data_broadcasts[(relevant_data_broadcasts['match_id'] == current_row['match_id']) & ((relevant_data_broadcasts['minute'] >= current_minute - size_window_before) & (relevant_data_broadcasts['minute'] <= current_minute + size_window_after))]
    window_content = ' '.join(window_data.content.tolist())
    
    # время окончания трансляции
    time_match = int(relevant_data_broadcasts[relevant_data_broadcasts['match_id'] == current_row['match_id']]['minute'].max())
    finish_match = relevant_data_broadcasts[relevant_data_broadcasts['match_id'] == current_row['match_id']].iloc[0]['match_time'] + dt.timedelta(minutes=time_match)
    
    for news_id in relevant_data_news[relevant_data_news['match_id'] == current_row['match_id']].index.tolist():
      # обобщающая новость
      after_match = predicted_data[predicted_data['id'] == relevant_data_news.loc[news_id]['id']]['result_match'].values[0]
      if after_match == 1:
        # только концовка
        finish_window_data = relevant_data_broadcasts[(relevant_data_broadcasts['match_id'] == current_row['match_id']) & (relevant_data_broadcasts['minute'] >= time_match - size_window_before)]
        finish_window_content = ' '.join(finish_window_data.content.tolist())

        # важные события с концовкой
        many_finish_window_data = relevant_data_broadcasts[(relevant_data_broadcasts['match_id'] == current_row['match_id']) & (relevant_data_broadcasts['minute'] < time_match) & relevant_data_broadcasts['type'].isin(content_type[1:9])]
        many_finish_window_content = ' '.join(many_finish_window_data.content.tolist())
        df['row_' + str(count_df_row)] = [current_row['match_id'], relevant_data_news.loc[news_id]['id'], current_row['match_time'], relevant_data_news.loc[news_id]['ctime'], after_match, current_row['name'], current_row['team1'], current_row['team2'], current_row['type'], many_finish_window_content + ' ' + finish_window_content, relevant_data_news.loc[news_id]['part_body']]
        count_df_row += 1
      # какое-то событие в матче
      else:
        df['row_' + str(count_df_row)] = [current_row['match_id'], relevant_data_news.loc[news_id]['id'], current_row['match_time'], relevant_data_news.loc[news_id]['ctime'], after_match, current_row['name'], current_row['team1'], current_row['team2'], current_row['type'], window_content, relevant_data_news.loc[news_id]['part_body']]
        count_df_row += 1

In [ ]:
dt_frame = pd.DataFrame.from_dict(df, orient='index', columns=['match_id', 'news_id', 'match_time', 'news_time', 'after_match', 'name', 'team1', 'team2', 'type', 'content', 'part_body']).drop_duplicates().reset_index().drop(['index'], axis=1)
dt_frame

,match_id,news_id,match_time,news_time,after_match,name,team1,team2,type,content,part_body
0,18925,7274151,2009-03-14 13:30:00,2009-03-14 17:48:18,0,Россия. Премьер-лига,Локомотив,Химки,yellowcard,23 минута. Родолфо после верховой борьбы с Ант...,Дюрица получил травму в игре с «Химками». Защи...
1,18925,7274964,2009-03-14 13:30:00,2009-03-14 19:12:30,0,Россия. Премьер-лига,Локомотив,Химки,yellowcard,23 минута. Родолфо после верховой борьбы с Ант...,Антипенко получил травму в матче с «Локомотиво...
2,18925,7274151,2009-03-14 13:30:00,2009-03-14 17:48:18,0,Россия. Премьер-лига,Локомотив,Химки,ball,"26 минута. ГООООООЛ! ""ХИМКИИИИ! Йованович, зам...",Дюрица получил травму в игре с «Химками». Защи...
3,18925,7274964,2009-03-14 13:30:00,2009-03-14 19:12:30,0,Россия. Премьер-лига,Локомотив,Химки,ball,"26 минута. ГООООООЛ! ""ХИМКИИИИ! Йованович, зам...",Антипенко получил травму в матче с «Локомотиво...
4,18925,7274151,2009-03-14 13:30:00,2009-03-14 17:48:18,0,Россия. Премьер-лига,Локомотив,Химки,zamena,44 минута. Травмированного Дюрицу меняет Финин...,Дюрица получил травму в игре с «Химками». Защи...
...,...,...,...,...,...,...,...,...,...,...,...
85627,1104865,1040593364,2016-05-31 00:00:00,2016-05-31 19:09:46,0,Кубок Стэнли,Питтсбург,Сан-Хосе,puck,"57 минута. ""Питтсбург"" поддавливает в концовке...",НХЛ. Марло не будет дисквалифицирован за удар ...
85628,1104866,1040621229,2016-06-02 00:00:00,2016-06-01 18:17:21,1,Кубок Стэнли,Питтсбург,Сан-Хосе,puck,29 минута. ГОООООООООООЛ!!! Кессел - 1:0! 10-й...,НХЛ. Кубок Стэнли. Финал. «Питтсбург» вновь по...
85629,1104866,1040636713,2016-06-02 00:00:00,2016-06-02 08:20:10,1,Кубок Стэнли,Питтсбург,Сан-Хосе,puck,29 минута. ГОООООООООООЛ!!! Кессел - 1:0! 10-й...,НХЛ. Форвард «Питтсбурга» Шери забил первый по...
85630,1104866,1040636981,2016-06-02 00:00:00,2016-06-02 08:41:16,1,Кубок Стэнли,Питтсбург,Сан-Хосе,puck,29 минута. ГОООООООООООЛ!!! Кессел - 1:0! 10-й...,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...


In [ ]:
dt_frame.to_csv('task4_pairs_data.csv', index=False, encoding='utf-8')
!cp /content/task4_pairs_data.csv "/content/drive/MyDrive/NIR/task4_pairs_data.csv"